In [175]:
import sys
import os

# 获取当前notebook的绝对路径
current_dir = os.path.abspath('')
# 获取项目根目录的路径（src的父目录）
project_root = os.path.dirname(os.path.dirname(current_dir))
sys.path.append(project_root)



### Get random properties as test set

In [176]:
import random
from typing import Any, Dict, List, Tuple
from src.utils.mongodb import get_collection


def get_random_properties(num_properties: int = 20) -> Tuple[List[Dict[str, Any]], List[int]]:
    collection = get_collection()
    
    # Get total count of documents
    total_docs = collection.count_documents({})
    
    # Generate random indices
    random_indices = random.sample(range(total_docs), min(num_properties, total_docs))
    
    # Get random documents
    properties = []
    ids = []
    for idx in random_indices:
        doc = collection.find().skip(idx).limit(1).next()
        properties.append({
            'id': doc.get('_id', ''),
            'name': doc.get('name', ''),
            'summary': doc.get('summary', ''),
            'description': doc.get('description', '')          
        })
        ids.append(doc.get('_id', ''))
    return properties, ids

In [177]:
random_properties, random_property_ids = get_random_properties(20)
random_properties[0], random_property_ids


({'id': 16281923,
  'name': '北角公寓 单人间',
  'summary': '北角 购物交通都很便利｡房间干净 自己住的房子',
  'description': '北角 购物交通都很便利｡房间干净 自己住的房子'},
 [16281923,
  26048057,
  1520042,
  14434534,
  32100680,
  27989463,
  9516607,
  3374964,
  28358504,
  28121603,
  7080271,
  1143151,
  3544610,
  8847051,
  152645,
  944736,
  13702313,
  17164830,
  6027345,
  1182475])

### Generate queries for these properties

In [178]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
user_llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.5)


def generate_query_for_property(property_info: Dict[str, str]) -> str:
    """Generate a query for a property using LLM."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that generates search queries for Airbnb properties."),
        ("human", """
        Based on the following Airbnb property information, generate a query that a potential guest might use to search for this property, such as "I want to stay in a house with a pool, a kitchen, and a balcony ...". 
        This query should be concise and corresponding to the property information, and should not be a question. Focus on the unique aspects mentioned in the summary. Query should between 30 and 50 words.
        The query should be in the same language as the summary.
        Summary: {summary}
        
        Query: """)
    ])
    
    chain = prompt | user_llm
    response = chain.invoke({
        "summary": property_info['summary']
    })
    return response.content.strip()

In [179]:
def generate_queries(properties: List[Dict[str, str]]) -> List[Dict[str, str]]:     
    # Generate queries for each property
    results = []
    for property_info in properties:
        query = generate_query_for_property(property_info)
        results.append({
            'id': property_info['id'],
            'name': property_info['name'],
            'summary': property_info['summary'],    
            'description': property_info['description'],
            'generated_query': query
        })
    return results

In [180]:
random_properties_and_queries = generate_queries(random_properties)

In [181]:
random_properties_and_queries

[{'id': 16281923,
  'name': '北角公寓 单人间',
  'summary': '北角 购物交通都很便利｡房间干净 自己住的房子',
  'description': '北角 购物交通都很便利｡房间干净 自己住的房子',
  'generated_query': '北角便利的购物和交通，干净的房间，适合自己入住的舒适住宿环境。'},
 {'id': 26048057,
  'name': 'The Tiny Boho',
  'summary': 'The Tiny Boho, um cantinho com terraço, situado a 3 minutos a pé da Rotunda da Boavista, a 300 metros da Casa da Música e a 400 metros do Mercado do Bom Sucesso. O Museu Nacional Soares dos Reis fica a 16 minutos a pé do apartamento, enquanto o Museu do Eléctrico está a 1,4 km. A 10 minutos de carro do museu Serralves e do Parque da cidade. O aeroporto situa-se a 9 km. É uma excelente escolha entre viajantes que estão interessados em arquitetura, caminhadas na cidade e degustar vinho.',
  'description': 'The Tiny Boho, um cantinho com terraço, situado a 3 minutos a pé da Rotunda da Boavista, a 300 metros da Casa da Música e a 400 metros do Mercado do Bom Sucesso. O Museu Nacional Soares dos Reis fica a 16 minutos a pé do apartamento, enquanto o Museu

In [182]:
sample_queries = [property['generated_query'] for property in random_properties_and_queries]
sample_queries

['北角便利的购物和交通，干净的房间，适合自己入住的舒适住宿环境。',
 'Procuro um apartamento com terraço perto da Rotunda da Boavista, Casa da Música e Mercado do Bom Sucesso, ideal para explorar arquitetura, fazer caminhadas urbanas e degustar vinho, com fácil acesso ao aeroporto e atrações culturais como museus e parques na cidade.',
 'Spacious 70 m2 apartment with 3 bedrooms, including a king size double room, two individual rooms, and two sofa beds, equipped with powerful AC, accessible via two lifts, located opposite a large commercial center for shopping and dining.',
 'Spacious apartment in a prestigious building in central Barcelona with concierge, near Rambla de Cataluña, accommodating up to 6 people, featuring 2 bedrooms with single beds, a large double bedroom, a fully equipped kitchen, living and dining areas, and two bathrooms.',
 'Comfortable 1-room apartment just 100m from the beach, walking distance to city center, supermarkets, restaurants, markets, shopping streets, and hospital, fully furnished, id

In [183]:
from typing import Optional
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from src.data_models import Address, ImageDescrib
import pandas as pd 

class SearchResultItem(BaseModel):
    id: int = Field(alias='_id')
    name: str
    accommodates: Optional[int] = None
    address: Address
    summary: Optional[str] = None
    description: Optional[str] = None
    neighborhood_overview: Optional[str] = None
    notes: Optional[str] = None
    images: ImageDescrib
    search_score: Optional[float] = None
    reviews: Optional[List[Dict[str, Any]]] = None

### Retrive top 10 most similar listings from database for each property

In [184]:

class FullTextSearch():
    def __init__(self, collection) -> None:
        self.collection = collection

    def _build_pipeline(self, query_text: str) -> list[dict]:
        pipeline = [
            {
                "$search": {
                    "index": "full_text_search_index", 
                    "text": {
                        "query": query_text,
                        "path": "description"
                    }
                }
            },
            {
                "$addFields": {
                    "search_score": {"$meta": "searchScore"}
                }
            },
            {
                "$sort": {
                    "search_score": -1
                }
            },
            {"$limit": 10}
        ]
        return pipeline

    def do_search(self, user_query: str) -> list[dict]:
        pipeline = self._build_pipeline(user_query)       
        return self.collection.aggregate(pipeline)

In [185]:
collection = get_collection()
fulltext_searcher = FullTextSearch(collection)


In [186]:
retrieved_results = []
for query in sample_queries:
    results = fulltext_searcher.do_search(query)
    search_results_models = [SearchResultItem(**result)  for result in results]
    search_results_df = pd.DataFrame([item.model_dump() for item in search_results_models])
    retrieved_results.append(search_results_df)

    
retrieved_results[0]['description']

0                              北角 购物交通都很便利｡房间干净 自己住的房子
1    香港铭城宾馆(家庭旅馆)(MAIN SHING HOTEL)位于铜锣湾地铁站附近,与利园一期...
2    特别干净和安静的楼房,独立卧室三菱冰箱松下微波炉全自动洗衣机家电齐全方便拎包入住｡地点位于中...
3    省!通過此連結創建您的帳戶獲取$ 250HKD優惠券作為您的第一次旅行住宿折扣卷  http...
4    此公寓为我们的明星房源,出自设计师之手,专业团队打造,全新装修,细节满溢心意｡ 适合商务出差...
5    我的房子在悉尼hills區castle hill,附近️️大型商場,游泳健身中心和club,...
6    公寓为酒店式公寓!位置优越!双口岸近会展!是赴港旅游和商务参展的首选目的地! 位置:位于福田...
7    这是一间温馨的小屋,拥有着三个房间,其中两间是屋主的私人空间｡还有一间拥有独立私密睡房是您的...
8    ★\t我有故事,你有酒吗? 【旅途路上的小栈】是我在这个城市中的一片静谧之地｡Sam是一位帅...
9    房间设施齐全,配备一个柔软舒适的1.5米双人床,舒适的四件套,适宜入住1-2个人｡房间整体设...
Name: description, dtype: object

In [187]:
retrieved_ids = []
for result in retrieved_results:
    retrieved_ids.append(result['id'].tolist())

### Check if random property ids are in the top k retrieved ids

In [188]:
top_k_positions = []
for i in range(len(retrieved_ids)):
    top_k_positions.append(check_top_k_positions(retrieved_ids[i], random_property_ids[i]))
    
top_k_positions

[array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([0, 0, 0, 0]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([0, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 1, 1]),
 array([0, 0, 0, 0])]

In [189]:
# Convert top_k_positions list to DataFrame
top_k_df = pd.DataFrame(top_k_positions, columns=['top_1', 'top_3', 'top_5', 'top_10'])
top_1 = sum(top_k_df['top_1'].tolist())/len(top_k_df)
top_3 = sum(top_k_df['top_3'].tolist())/len(top_k_df)
top_5 = sum(top_k_df['top_5'].tolist())/len(top_k_df)
top_10 = sum(top_k_df['top_10'].tolist())/len(top_k_df)

top_1, top_3, top_5, top_10

(0.85, 0.9, 0.9, 0.9)

### Using Ragas to evaluate the response by LLM
- Context Recall 
- Faithfulness
- Factual correctness

Context Recall measures how many of the relevant documents (or pieces of information) were successfully retrieved. It focuses on not missing important results. Higher recall means fewer relevant documents were left out. In short, recall is about not missing anything important. Since it is about not missing anything, calculating context recall always requires a reference to compare against.

The Faithfulness metric measures how factually consistent a response is with the retrieved context. It ranges from 0 to 1, with higher scores indicating better consistency.

FactualCorrectness is a metric class that evaluates the factual correctness of responses generated by a language model. It uses claim decomposition and natural language inference (NLI) to verify the claims made in the responses against reference texts.

In [190]:
expected_responses = []
for i in range(len(sample_queries)):
    expected_responses.append(random_properties[i]['description'])


In [191]:
top_1_retrieved_contexts = []
top_1_retrieved_contexts = [retrieved_results[i]['description'][0] for i in range(len(retrieved_results))]
top_1_retrieved_contexts


['北角 购物交通都很便利｡房间干净 自己住的房子',
 'The Tiny Boho, um cantinho com terraço, situado a 3 minutos a pé da Rotunda da Boavista, a 300 metros da Casa da Música e a 400 metros do Mercado do Bom Sucesso. O Museu Nacional Soares dos Reis fica a 16 minutos a pé do apartamento, enquanto o Museu do Eléctrico está a 1,4 km. A 10 minutos de carro do museu Serralves e do Parque da cidade. O aeroporto situa-se a 9 km. É uma excelente escolha entre viajantes que estão interessados em arquitetura, caminhadas na cidade e degustar vinho. Estamos disponíveis para qualquer dúvida ou informação. Quer por telefone ou (Website hidden by Airbnb) supermercado em frente ao apartamento, centro comercial a 3 minutos a pé. Zona com todo o tipo de transportes públicos disponíveis. Super central, com tudo o que precisa, está a 5minutos de metro do centro histórico da cidade. Estacionamento pago na rua. Estação de metro "Casa da Música" a 2min de distância a pé.',
 "-  70 m2       -  3 bedrooms      - 1 Double Room , king 

In [192]:
rag_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


def generate_response(query, context):      
    prompt_template = ChatPromptTemplate.from_messages([
                ("system", """You are an Airbnb listing recommendation system. Please:
                1. Respond in the same language as the user
                2. If the user is asking for property recommendations:
                   - Prioritize results with higher search scores
                   - Include the Airbnb listing URL and image URL
                   - Explain why you chose these properties
                   - Highlight features that match the user's criteria
                3. If the user has provided an image, consider visual similarity in your recommendations
                4. Be friendly and helpful in your responses
                5, answer the question in the same language as the query"""),
                ("human", "Answer this user query: {query} with the following context:\n{context}")
            ])

    formatted_messages = prompt_template.format_messages(query=query, context=context)
    return rag_llm(formatted_messages)  

In [193]:
dataset = []

for query, reference, context in zip(sample_queries, expected_responses, top_1_retrieved_contexts):
    response = generate_response(query, context)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts": [context],
            "response":response.content,
            "reference":reference
        }
    )

In [194]:
dataset

[{'user_input': '北角便利的购物和交通，干净的房间，适合自己入住的舒适住宿环境。',
  'retrieved_contexts': ['北角 购物交通都很便利｡房间干净 自己住的房子'],
  'response': '根据您的需求，我为您推荐了几处在北角的舒适住宿，适合自己入住，并且购物和交通都非常便利。\n\n1. **北角现代公寓**\n   - **链接**: [北角现代公寓](https://www.airbnb.com/rooms/12345678)\n   - **图片**: ![北角现代公寓](https://a.airbnb.com/12345678.jpg)\n   - **推荐理由**: 这间公寓拥有现代化的设计，房间干净整洁，适合单人入住。距离购物中心和地铁站都很近，出行非常方便。\n\n2. **北角舒适单间**\n   - **链接**: [北角舒适单间](https://www.airbnb.com/rooms/87654321)\n   - **图片**: ![北角舒适单间](https://a.airbnb.com/87654321.jpg)\n   - **推荐理由**: 这间单间提供了一个温馨的居住环境，房间内设施齐全，适合独自旅行者。周围有许多便利店和餐馆，生活非常方便。\n\n3. **北角时尚公寓**\n   - **链接**: [北角时尚公寓](https://www.airbnb.com/rooms/23456789)\n   - **图片**: ![北角时尚公寓](https://a.airbnb.com/23456789.jpg)\n   - **推荐理由**: 这间公寓设计时尚，空间宽敞，干净舒适。步行即可到达购物区和公共交通站，方便您出行。\n\n这些推荐的住宿都符合您的要求，提供了干净舒适的环境，并且在北角的购物和交通都非常便利。希望您能找到满意的住宿！如果您有其他问题或需要更多建议，请随时告诉我！',
  'reference': '北角 购物交通都很便利｡房间干净 自己住的房子'},
 {'user_input': 'Procuro um apartamento com terraço perto da Rotunda da Boavista, Casa da Música e Merc

In [195]:
from ragas import EvaluationDataset
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness


evaluation_dataset = EvaluationDataset.from_list(dataset)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))

In [196]:
result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result


Evaluating:  52%|█████▏    | 31/60 [00:15<00:08,  3.59it/s]Exception raised in Job[17]: OutputParserException(Invalid json output: {
    "claims": [
      "The property is located on the waterfront on the canal.",
      "Guests can wake up on the water.",
      "Guests can walk to the beach.",
      "The property is located just outside of downtown Kailua.",
      "Kailua is a beach town situated along one of the best stretches of sand in the country.",
      "The home is a two-story house.",
      "The house has been split to make a comfortable place for guests.",
      "The unit is directly above a second AirBnB unit.",
      "The second AirBnB unit may or may not be rented during the guest's stay.",
      "Guests are encouraged to rent both units with their best friend.",
      "The welcome phrase "E Komo Mai" means "Welcome".",
      "The hosts hope guests have the best time staying and playing in Hawaii.",
      "The hosts hope guests leave with many fond memories of their time on

{'context_recall': 0.9114, 'faithfulness': 0.6587, 'factual_correctness(mode=f1)': 0.6032}

In [197]:
result.upload() 

[2025-04-15 21:00:26 - (2025-04-16 01:00:26 UTC)] [ERROR] [ragas.utils] [RagasID: a-d3ee32eef0fe470fa68b390f1d1bfd91, App-Version: 0.2.14] [API_ERROR] Request failed. Status Code: 500, URL: https://api.ragas.io/api/v1/alignment/evaluation, Error Message: 
API Message: An internal server error occured


UploadException: Request failed: 
API Message: An internal server error occured